In [ ]:
import os, sys, json
import csv
import string
from tqdm import tqdm
import re
from collections import deque
from difflib import SequenceMatcher
from bs4 import BeautifulSoup
import spacy

import codecs

In [ ]:
movie_list = {}

movie_list_filename = 'book_lists/storedScript.csv'
with open(movie_list_filename, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',', quotechar='\"')
    for i, row in enumerate(csvreader):
        if len(row) != 2:
            print('Error: different length:', len(row), row)
        if i == 0:
            continue
        movie_list[row[1]] = 1
    csvfile.close()
print(movie_list)

# spacy language corpus
NLP_PIPE = spacy.load("en_core_web_sm")

exclusive_id = [
    '39b1f44ec2639226b6025140e2a7e7eaf570409e',     # content format issue
    '492f2d56eba93816e7d0958e2ba62d36d93bc97e',     # content format issue
    '0269408ac78193a0e13f1033bec6f658b00437b3',     # content format issue
    '0bc7352d6a0e678c0d8acc57c0c1cc3466fe9ef7',     # no <b></b>
    '405df1ccf0409ea6040c2a765a0315878f991d79',     # no <b></b>
    '4112d61d9880d57229b3a49a5d04e0bc534e44a8',     # no    </b>
    '823a8b6bbbb0bbeec9c4bec1929b64a3694232d7',     # no <b></b>
    '1aae28477e771b3af008ec59ce29086a1bc66776',     # download failed?
    '3747036f950fe8f79cdaa0eb713104b9eb8af6c5',     # download failed?
    '5283fa0a6ea69f4b4224d12018bbf985a2b80543',     # download failed?
]

special_id = [
    '08172f313a4053b150a6c20310b6ee58f6968b76',     # no <b></b>
    '37c11f984cb14401d85abfc20e8305ca7a472c9f',     # no <b></b>
]

STORY_META = dict()
with open('documents.csv','r') as fp:
    content = csv.reader(fp, delimiter=',')
    for each_line in content:
        if content.line_num == 1:
            print('column ==>', each_line)
            continue
        # 0: 'document_id', 
        # 1: 'set', 
        # 2: 'kind', 
        # 3: 'story_url', 
        # 4: 'story_file_size', 
        # 5: 'wiki_url', 
        # 6: 'wiki_title', 
        # 7: 'story_word_count'
        # 8: 'story_start'
        # 9: 'story_end'
        
        if each_line[0] not in movie_list:
            continue
        
        STORY_META[each_line[0]] = dict()
        STORY_META[each_line[0]]['set']             = each_line[1]
        STORY_META[each_line[0]]['kind']            = each_line[2]
        STORY_META[each_line[0]]['story_url']       = each_line[3]
        STORY_META[each_line[0]]['story_file_size'] = each_line[4]
        STORY_META[each_line[0]]['wiki_url']        = each_line[5]
        STORY_META[each_line[0]]['wiki_title']      = each_line[6]
        STORY_META[each_line[0]]['story_word_count'] = each_line[7]
        STORY_META[each_line[0]]['story_start']     = each_line[8]
        STORY_META[each_line[0]]['story_end']       = each_line[9]

print(len(movie_list))
print(len(STORY_META))

{'a13cfb713f7eca7e750b8ad20b946b142aaa5dbf': 1, 'e1c042c57411f230068ededfd7b27e44c0580700': 1, '0bc7352d6a0e678c0d8acc57c0c1cc3466fe9ef7': 1, '8928a603a3c6d154ea4547060805a966ce0f4d60': 1, '7ef394b11230baf81f61e790839fa993c6ea1f72': 1, 'ea5d07dd2150a3e4fd5199ab496074839a019ded': 1, '91293cd81f021de45cffd363ef81dd95d2c122a3': 1, '71ce19cf034c830c1e2d8b98682ca1d53ead1067': 1, '51f52296a99ce50b6ca69aa5939d81a5aea4e042': 1, 'c72161c89a7dc8ea5d62b200689bd2acae6f354d': 1, 'bfd50b1cf73709308ab4ad727d829c5fca23480c': 1, 'ef722cf82033c8e66197209f06a9cb9754be78d9': 1, '214cb5277750c1ccddec0b10fa545c3c78c45f64': 1, '403479540337f035a011f0ea84b4770e31e18207': 1, '1ddea1c3f13183fbe029834b5a3ab505600c5dfa': 1, 'ccfabbee5a88f09447f3cbba5ea0f86e5c039058': 1, 'dae7e31c643a625b54fd042a8ecad0d9e964494e': 1, '81db4f3e5fe29c02fd7b7a702aa4847db6a04613': 1, 'dcd3468c3e18822f08ff0607edfb3e9ca06f3ef0': 1, '3b2427be8d3bbfe55590d1ce307f778ede5bbcd9': 1, 'f6de97a4d111d0663b747eb10e123952424786d0': 1, '4a1fff119d0

column ==> ['document_id', 'set', 'kind', 'story_url', 'story_file_size', 'wiki_url', 'wiki_title', 'story_word_count', 'story_start', 'story_end']
442
442


In [ ]:
#############################################
##### getting split meta-info via stats #####
#############################################

count_movie = 0
count_meta_complete = 0
count_start_pattern = 0
count_summary = 0
if not os.path.exists('./preprocessed/'):
    os.mkdir('./preprocessed/')
if not os.path.exists('./preprocessed/movies/'):
    os.mkdir('./preprocessed/movies/')
    
count_with_fade_in = 0
list_num_scenes = []
list_num_dialogs = []

num_verified = 0
num_failed = 0
num_movie = 0
num_fade_in_success = 0

meta_fileout = open('preprocessed/split_meta_info.txt', 'w')
    
for each_id in STORY_META.keys():
    if each_id in exclusive_id:
        print(each_id, 'has some issues and will be skipped')
        continue
        
    if each_id in special_id:
        print(each_id, 'currently skipping the special script formats')
        continue
#     each_id = 'cd13eb380843224f66c34047cc06dc445a92f8fd'
#     each_id = '18bcdf16fb896faf7fb36db6b752f16f0343ad24'
#     each_id = '1508452704b5829941b98aeee2eeba192f5beae5'
#     each_id = '08373775c3d82d8bbf4ff286a3a0076a3627c652'
    print(each_id)
    num_movie += 1
    
    num_scenes = 0
    num_dialogs = 0
    
    with open(os.path.join('tmp', each_id + '.content'), 'r', encoding='cp1256') as file:
        content = file.read()

    soup = BeautifulSoup(content, 'html.parser')
    
    bs = soup.pre.find_all('b')

#     result = [[element.string, element.next_sibling.string] for element in bs if (element.next_sibling and element.next_sibling.string is not None)]

    sents = [(element.string, [ele for ele in element.next_sibling.string.split('\n') if ele.strip()]) #split the text by \n; remove space
             for element in bs if (element.next_sibling and element.next_sibling.string is not None)] #select text 
    
    scene_symbol = None
    dialogue_symbol = None
    scene_found = False
    fade_in_indent = -1

    for element in bs:
#         print(element.string)
        if element is None or element.string is None:
            continue
        if not scene_found and element.string.find('FADE IN') != -1:#find if there is a FADE IN in the sentence
            scene_symbol = str(element.string)[:element.string.index('FADE IN')]#count the space before FADE IN
            scene_found = True#only need the 1st FADE IN (incase there is any FADE IN in the text)

        if scene_found and element.string.startswith(scene_symbol + scene_symbol):#find if there are texts start with 2* the space before FADE IN
            dialogue_symbol = scene_symbol + scene_symbol
            break

    if scene_symbol is None:# there is no FADE IN in the script
        print("didn't find the symbol for scenes")
    else:
        print([scene_symbol])
        
        pointer = 0
        if pointer != len(scene_symbol):
            while scene_symbol[pointer] == ' ' or scene_symbol[pointer] == '\t' or scene_symbol[pointer] == '\n':
                pointer += 1
                if pointer == len(scene_symbol):
                    break
        fade_in_indent = pointer
        print('FADE IN indent:', fade_in_indent)
#         fade_in_indent = len(scene_symbol)
        count_with_fade_in += 1
    
    meta_info = {}
    meta_info['id'] = each_id
    if scene_symbol is None:
        meta_info['fade-in symbol'] = []
    else:
        meta_info['fade-in symbol'] = [scene_symbol]
    meta_info['fade-in indent'] = fade_in_indent
    
#     meta_fileout.write('id\t%s\n'%each_id)
#     if scene_symbol is None:
#         meta_fileout.write('%s\n'%json.dumps([]))
#     else:
#         meta_fileout.write('%s\n'%json.dumps([scene_symbol]))
#     meta_fileout.write('fade-in indent\t%d\n'%fade_in_indent)
        
    scene_indent_dict = {}
    indent_dict = {}
    dialog_indent_dict = {}
    section_structures = []
    
    new_sections = []
#     num_movie += 1

    for section in sents:
#         section_name = re.sub(r'\t', '', section[0]).strip()
#     #     section_name = re.sub(r'\t', '', section[0]).strip()
#         section_texts = ''

        prev_indent = -1
        new_scene = False
        
        for sent in section[1]:
            pointer = 0
            if pointer == len(sent):
    #             section_structures.append([section_name, '', ''])
                continue
            while sent[pointer] == ' ' or sent[pointer] == '\t' or sent[pointer] == '\n':
                pointer += 1
                if pointer == len(sent):
                    break

            if pointer == len(sent):
    #             section_structures.append([section_name, '', ''])
                continue

            indent = pointer
    #         if indent > 30:
    #             print(section)
    #         section_type = indent
    #         section_content = section_content[pointer:]
    
#             if fade_in_indent == -1:
            if indent not in indent_dict:
                indent_dict[indent] = 1
            else:
                indent_dict[indent] += 1
    
            if fade_in_indent != -1:
                if indent > fade_in_indent:
                    if indent not in dialog_indent_dict:
                        dialog_indent_dict[indent] = 1
                    else:
                        dialog_indent_dict[indent] += 1
                    num_dialogs += 1
                else:
                    if indent not in scene_indent_dict:
                        scene_indent_dict[indent] = 1
                    else:
                        scene_indent_dict[indent] += 1
                    num_scenes += 1
                    
#                 if indent <= fade_in_indent and prev_indent > fade_in_indent:
#                     new_sections.append(('DIALOG', section_name, section_texts))
#                     num_dialogs += 1
#                     section_name = 'NULL'
#                     section_texts = ''
#                     new_scene = True

#         #         print(sent)
#                 section_texts += re.sub(r'\t', '', sent).strip() + '\n'
#                 prev_indent = indent
               
#         if fade_in_indent != -1:
#             if new_scene:
#                 new_sections.append(('SCENE', section_name, section_texts))
#                 num_scenes += 1
#             else:
#                 if prev_indent > fade_in_indent:
#                     new_sections.append(('DIALOG', section_name, section_texts))
#                     num_dialogs += 1
#                 else:
#                     new_sections.append(('SCENE', section_name, section_texts))
#                     num_scenes += 1
    if len(indent_dict) == 0:
        print('Empty indent dict', indent_dict)
        meta_info['type'] = 'Fail'
        meta_info['indent_dict'] = indent_dict
#         meta_fileout.write('Fail\n')
#         meta_fileout.write(json.dumps(indent_dict) + '\n\n')
    elif fade_in_indent == -1:
        print('OVERALL indents:', indent_dict)
        
        sorted_indents = sorted(indent_dict.items(), key=lambda x:x[0])
        num_total = 0
        for k, v in indent_dict.items():
            num_total += v
            
        scene_indent_dict = {}
        dialog_indent_dict = {}
        cur_num_scenes = 0
        adding_scene = True
        failure = False
        split_num = 0
        for idx, indent_pair in enumerate(sorted_indents):
            cur_ratio = cur_num_scenes / num_total
            next_ratio = (cur_num_scenes + indent_pair[1]) / num_total
            
            if adding_scene:
                print(cur_ratio, next_ratio)
            
            # a huge jump refers to failure case
            if cur_ratio <= 0.3870662796573337 - 0.13559559099176607 and next_ratio >= 0.60: # 0.3870662796573337 + 0.13559559099176607:
                failure = True
                break
            
            if adding_scene and next_ratio >= 0.6: #0.3870662796573337 + 0.13559559099176607: # split here
                adding_scene = False
                split_num = indent_pair[0]
                print('split here')
            if adding_scene:
                scene_indent_dict[indent_pair[0]] = indent_pair[1]
                cur_num_scenes += indent_pair[1]
            else:
                dialog_indent_dict[indent_pair[0]] = indent_pair[1]
        if not failure:
            print('!!!Re-organized overall dicts:')
            print('SCENE indents:', scene_indent_dict)
            print('DIALOG indents:', dialog_indent_dict)
            print('Verified via ratio', 'split at', split_num)
            num_verified += 1
#             meta_fileout.write('Reorg overall\n')
#             meta_fileout.write(json.dumps(indent_dict) + '\n\n')
            meta_info['type'] = 'Reorg overall'
            meta_info['indent_dict'] = indent_dict
            meta_info['scene_dict'] = scene_indent_dict
            meta_info['dialog_dict'] = dialog_indent_dict
        else:
            print('!!!Failed to split (with overall)')
            num_failed += 1
#             meta_fileout.write('Fail\n')
#             meta_fileout.write(json.dumps(indent_dict) + '\n\n')
            meta_info['type'] = 'Fail'
            meta_info['indent_dict'] = indent_dict
        
    else:
        print('SCENE indents:', scene_indent_dict)
        print('DIALOG indents:', dialog_indent_dict)
        list_num_scenes.append(num_scenes)
        list_num_dialogs.append(num_dialogs)
        
        ratio_scene = num_scenes / (num_scenes + num_dialogs)
        if ratio_scene <= 0.6 and ratio_scene >= 0.3870662796573337 - 0.13559559099176607:
            print('!!!Original dicts:')
            print('Verified via ratio')
            num_verified += 1
            num_fade_in_success += 1
#             meta_fileout.write('Original fade-in\n')
#             meta_fileout.write(json.dumps(indent_dict) + '\n\n')
            meta_info['type'] = 'Original fade-in'
            meta_info['indent_dict'] = indent_dict
            meta_info['scene_dict'] = scene_indent_dict
            meta_info['dialog_dict'] = dialog_indent_dict
        else:
            sorted_indents = sorted(indent_dict.items(), key=lambda x:x[0])
            num_total = 0
            for k, v in indent_dict.items():
                num_total += v

            scene_indent_dict = {}
            dialog_indent_dict = {}
            cur_num_scenes = 0
            adding_scene = True
            failure = False
            split_num = 0
            for idx, indent_pair in enumerate(sorted_indents):
                cur_ratio = cur_num_scenes / num_total
                next_ratio = (cur_num_scenes + indent_pair[1]) / num_total
                
                if adding_scene:
                    print(cur_ratio, next_ratio)

                # a huge jump refers to failure case
                if cur_ratio <= 0.3870662796573337 - 0.13559559099176607 and next_ratio >= 0.6: # 0.3870662796573337 + 0.13559559099176607:
                    failure = True
                    break

                if adding_scene and next_ratio >= 0.6: # 0.3870662796573337 + 0.13559559099176607: # split here
                    adding_scene = False
                    split_num = indent_pair[0]
                    print('split here')
                if adding_scene:
                    scene_indent_dict[indent_pair[0]] = indent_pair[1]
                    cur_num_scenes += indent_pair[1]
                else:
                    dialog_indent_dict[indent_pair[0]] = indent_pair[1]
            if not failure:
                print('!!!Re-organized dicts:')
                print('SCENE indents:', scene_indent_dict)
                print('DIALOG indents:', dialog_indent_dict)
                print('Verified via ratio', 'split at', split_num)
                num_verified += 1
#                 meta_fileout.write('Reorg fade-in\n')
#                 meta_fileout.write(json.dumps(scene_indent_dict) + '\n\n')
#                 meta_fileout.write(json.dumps(dialog_indent_dict) + '\n\n')
                meta_info['type'] = 'Reorg fade-in'
                meta_info['indent_dict'] = indent_dict
                meta_info['scene_dict'] = scene_indent_dict
                meta_info['dialog_dict'] = dialog_indent_dict
            else:
                print('!!!Failed to split')
                num_failed += 1
#                 meta_fileout.write('Fail\n')
#                 meta_fileout.write(json.dumps(indent_dict) + '\n\n')
                meta_info['type'] = 'Fail'
                meta_info['indent_dict'] = indent_dict

    meta_fileout.write(json.dumps(meta_info) + '\n')
#     break
meta_fileout.close()


0025577043f5090cd603c6aea60f26e236195594
didn't find the symbol for scenes
Empty indent dict {}
00ee9e01a0e581e0d8cbf7e865a895147c480c5e
['                                                                    ']
FADE IN indent: 68
SCENE indents: {39: 97, 30: 156, 59: 1, 68: 10, 14: 25, 65: 15, 10: 645, 15: 691, 16: 571, 38: 85, 32: 21, 28: 160, 67: 10, 25: 86, 35: 55, 37: 24, 29: 154, 36: 108, 66: 15, 33: 161, 27: 50, 23: 276, 34: 26, 11: 55, 26: 174, 40: 24, 17: 31, 31: 45, 21: 40, 20: 249, 62: 5, 24: 37, 63: 10, 61: 1, 60: 4, 42: 18, 43: 1, 41: 20, 47: 1, 12: 1, 18: 2, 22: 4}
DIALOG indents: {77: 13, 82: 3, 80: 7, 71: 33, 76: 19, 72: 6, 75: 18, 70: 20, 78: 5, 74: 13, 73: 23, 69: 12, 79: 8, 84: 1, 83: 1}
0.0 0.14841233317993557
0.14841233317993557 0.16106764841233318
0.16106764841233318 0.16129774505292221
0.16129774505292221 0.16705016106764842
0.16705016106764842 0.3260469397146802
0.3260469397146802 0.45743212149102624
0.45743212149102624 0.4645651173492867
0.4645651173492867 0.46502

['          ']
FADE IN indent: 10
SCENE indents: {10: 3035}
DIALOG indents: {38: 1, 37: 1, 27: 1, 11: 183}
0.0 0.9422539583980131
!!!Failed to split
0fe1c42d024d1d3f001a9aadb400512ecbda2b9f
['']
FADE IN indent: 0
SCENE indents: {0: 640}
DIALOG indents: {27: 1, 18: 1, 12: 1663, 19: 34}
!!!Original dicts:
Verified via ratio
0ffdd0e4abb67f0012a68d719a2d509ee8ec643d
didn't find the symbol for scenes
OVERALL indents: {0: 1725, 7: 31, 10: 1261, 14: 295, 15: 91, 13: 220, 17: 69, 18: 6, 16: 112, 12: 346, 8: 30, 11: 225, 9: 9, 6: 57, 3: 72, 20: 1, 19: 4, 22: 1, 5: 31}
0.0 0.37614478848669863
0.37614478848669863 0.3918447448757087
0.3918447448757087 0.3986044483209769
0.3986044483209769 0.4110335804622765
0.4110335804622765 0.4177932839075447
0.4177932839075447 0.42433493240296555
0.42433493240296555 0.4262974269515918
0.4262974269515918 0.7012647187091147
split here
!!!Re-organized overall dicts:
SCENE indents: {0: 1725, 3: 72, 5: 31, 6: 57, 7: 31, 8: 30, 9: 9}
DIALOG indents: {10: 1261, 11: 22

['               ']
FADE IN indent: 15
SCENE indents: {15: 1419}
DIALOG indents: {39: 2, 57: 1, 25: 2121, 30: 248}
!!!Original dicts:
Verified via ratio
1aae28477e771b3af008ec59ce29086a1bc66776 has some issues and will be skipped
1ae015eaa8cd3b66136578e947afc72ed338fe10
['          ']
FADE IN indent: 10
SCENE indents: {10: 1392}
DIALOG indents: {35: 1, 32: 1, 11: 1036, 26: 19, 25: 23}
!!!Original dicts:
Verified via ratio
1b48982220584874f6abed587219f6d73e85abd8
didn't find the symbol for scenes
OVERALL indents: {30: 1, 20: 1131, 22: 70, 4: 30, 14: 62, 21: 344, 53: 6, 55: 19, 0: 117, 19: 117, 60: 32, 26: 3, 59: 8, 5: 511, 23: 42, 15: 738, 56: 8, 28: 53, 27: 99, 61: 3, 58: 23, 33: 6, 54: 17, 57: 7, 29: 6, 24: 5, 16: 281, 6: 169, 34: 2, 32: 1, 17: 19, 18: 27}
0.0 0.029567854435178165
0.029567854435178165 0.037149355572403335
0.037149355572403335 0.1662875916098054
0.1662875916098054 0.20899671468284053
0.20899671468284053 0.22466515036643922
0.22466515036643922 0.4111700783421784
0.41117

didn't find the symbol for scenes
OVERALL indents: {31: 1, 19: 1, 0: 1744, 10: 5368, 15: 1080, 11: 1}
0.0 0.21281269066503966
0.21281269066503966 0.8678462477120196
!!!Failed to split (with overall)
265fa34cf1f2c6145cce7ee9402bb3af6d898624
didn't find the symbol for scenes
OVERALL indents: {36: 2, 10: 3698, 11: 16}
0.0 0.9951560818083961
!!!Failed to split (with overall)
266f5b2295980ad31b5090a6c51b69055c87b3a7
['                                                             ']
FADE IN indent: 61
SCENE indents: {14: 30, 49: 2, 4: 84, 18: 139, 10: 25, 0: 32, 15: 484, 17: 231, 12: 30, 13: 20, 9: 38, 24: 36, 29: 4, 20: 78, 5: 631, 19: 45, 23: 17, 16: 738, 7: 19, 22: 53, 21: 104, 6: 306, 2: 1, 30: 1, 27: 1, 8: 5, 25: 7, 45: 1, 11: 1, 26: 2, 1: 1}
DIALOG indents: {73: 1}
0.0 0.010104199557941269
0.010104199557941269 0.010419955794126934
0.010419955794126934 0.0107357120303126
0.0107357120303126 0.03725923586990843
0.03725923586990843 0.23650142090306284
0.23650142090306284 0.3331228291758762


['\t']
FADE IN indent: 1
SCENE indents: {1: 1546}
DIALOG indents: {2: 995, 3: 78, 4: 12}
!!!Original dicts:
Verified via ratio
35f4e96cb364cf2204c611a94ca3e42511576ecc
didn't find the symbol for scenes
OVERALL indents: {5: 124, 30: 49, 20: 803}
0.0 0.12704918032786885
0.12704918032786885 0.9497950819672131
!!!Failed to split (with overall)
361364f57460139410c4130a1e7a58caf152c2bd
['    ']
FADE IN indent: 4
SCENE indents: {4: 124, 0: 320}
DIALOG indents: {34: 1, 28: 1, 14: 140, 18: 110, 19: 140, 24: 3, 17: 42, 5: 579, 15: 580, 20: 68, 22: 20, 16: 644, 21: 21, 13: 33, 10: 708, 11: 25, 6: 533, 23: 12, 12: 24, 26: 2, 25: 1}
0.0 0.0774630839990317
0.0774630839990317 0.1074800290486565
0.1074800290486565 0.2476397966594045
0.2476397966594045 0.3766642459452917
0.3766642459452917 0.5480513192931493
0.5480513192931493 0.5541031227305737
0.5541031227305737 0.5599128540305011
0.5599128540305011 0.5679012345679012
0.5679012345679012 0.6017913338174776
split here
!!!Re-organized dicts:
SCENE inden

...

didn't find the symbol for scenes
OVERALL indents: {0: 22}
0.0 1.0
!!!Failed to split (with overall)
c0e05b3aac173e39064f07ccb60fb0a30430f824
didn't find the symbol for scenes
OVERALL indents: {21: 236, 7: 1288, 17: 1836, 22: 24}
0.0 0.3806146572104019
0.3806146572104019 0.9231678486997635
split here
!!!Re-organized overall dicts:
SCENE indents: {7: 1288}
DIALOG indents: {17: 1836, 21: 236, 22: 24}
Verified via ratio split at 17
c29a505a7f2efe48bc73da66d4608888b8dc617b
didn't find the symbol for scenes
OVERALL indents: {34: 1, 24: 1, 23: 1, 29: 1, 57: 1, 10: 3211, 11: 24}
0.0 0.9910493827160494
!!!Failed to split (with overall)
c2e872845e99f82ddfa5a3ed84f985a04b6729e6
['\t\t']
FADE IN indent: 2
SCENE indents: {}
DIALOG indents: {12: 1529, 22: 1845, 26: 56}
0.0 0.4457725947521866
0.4457725947521866 0.9836734693877551
split here
!!!Re-organized dicts:
SCENE indents: {12: 1529}
DIALOG indents: {22: 1845, 26: 56}
Verified via ratio split at 22
c3d05fedec86ea11bb70837d54b941589bde4d88
didn'

['\t']
FADE IN indent: 1
SCENE indents: {0: 16, 1: 1390}
DIALOG indents: {17: 1, 15: 2, 19: 1, 16: 1, 18: 1, 42: 1, 27: 1, 36: 1, 6: 1, 3: 167, 2: 1715, 4: 10, 10: 1, 5: 1}
!!!Original dicts:
Verified via ratio
ce8cb184a11535e7a7c824c82b7772a1c3a7c92c
didn't find the symbol for scenes
OVERALL indents: {33: 1, 30: 1, 10: 751, 11: 1229, 26: 41, 25: 1}
0.0 0.37104743083003955
0.37104743083003955 0.9782608695652174
split here
!!!Re-organized overall dicts:
SCENE indents: {10: 751}
DIALOG indents: {11: 1229, 25: 1, 26: 41, 30: 1, 33: 1}
Verified via ratio split at 11
cf09f10dcf0a2d000a7d2a0fc776f16a0e5a86be
['          ']
FADE IN indent: 10
SCENE indents: {10: 3056}
DIALOG indents: {34: 2, 35: 1, 53: 1, 11: 3}
0.0 0.9977146588312112
!!!Failed to split
cf873fb685ac6b1bd09c733ad9b0c0130d109454
['\t']
FADE IN indent: 1
SCENE indents: {1: 864}
DIALOG indents: {2: 1660, 4: 96, 3: 175}
!!!Original dicts:
Verified via ratio
d03d0175a1b8416123ba9ee27fd22eda45e5795d
didn't find the symbol for scenes

didn't find the symbol for scenes
OVERALL indents: {23: 1, 22: 1, 42: 1, 0: 987, 10: 1751, 14: 97, 15: 6, 4: 4, 56: 2, 12: 154, 17: 2, 16: 6, 60: 1, 13: 28, 47: 1, 1: 1}
0.0 0.3243509694380545
0.3243509694380545 0.32467959250739403
0.32467959250739403 0.3259940847847519
0.3259940847847519 0.9014130791981597
split here
!!!Re-organized overall dicts:
SCENE indents: {0: 987, 1: 1, 4: 4}
DIALOG indents: {10: 1751, 12: 154, 13: 28, 14: 97, 15: 6, 16: 6, 17: 2, 22: 1, 23: 1, 42: 1, 47: 1, 56: 2, 60: 1}
Verified via ratio split at 10
d9df8732f4fad8d4ffa6d8b2f7af12ea374a5be2
didn't find the symbol for scenes
OVERALL indents: {26: 3, 25: 3, 45: 1, 4: 14, 10: 501, 61: 12, 0: 924, 17: 37, 13: 673, 59: 6, 62: 6, 60: 19, 58: 20, 18: 3, 14: 33, 53: 1, 56: 3, 63: 5, 15: 1, 16: 4, 19: 6, 64: 2, 20: 2, 11: 19, 57: 5, 23: 1, 55: 2, 24: 1, 12: 18}
0.0 0.39741935483870966
0.39741935483870966 0.40344086021505376
0.40344086021505376 0.6189247311827957
split here
!!!Re-organized overall dicts:
SCENE indents:

didn't find the symbol for scenes
OVERALL indents: {26: 1, 25: 5, 45: 1, 0: 106, 4: 49, 14: 24, 64: 1, 17: 128, 59: 31, 58: 28, 18: 727, 5: 926, 32: 6, 15: 216, 60: 9, 61: 5, 23: 15, 51: 1, 57: 27, 53: 2, 24: 11, 56: 14, 16: 275, 20: 147, 6: 615, 55: 1, 62: 3, 19: 473, 7: 193, 52: 1, 33: 1, 21: 3, 22: 2, 63: 1}
0.0 0.026185770750988144
0.026185770750988144 0.038290513833992096
0.038290513833992096 0.26704545454545453
0.26704545454545453 0.4189723320158103
0.4189723320158103 0.4666501976284585
0.4666501976284585 0.47257905138339923
0.47257905138339923 0.5259387351778656
0.5259387351778656 0.5938735177865613
0.5938735177865613 0.6254940711462451
split here
!!!Re-organized overall dicts:
SCENE indents: {0: 106, 4: 49, 5: 926, 6: 615, 7: 193, 14: 24, 15: 216, 16: 275}
DIALOG indents: {17: 128, 18: 727, 19: 473, 20: 147, 21: 3, 22: 2, 23: 15, 24: 11, 25: 5, 26: 1, 32: 6, 33: 1, 45: 1, 51: 1, 52: 1, 53: 2, 55: 1, 56: 14, 57: 27, 58: 28, 59: 31, 60: 9, 61: 5, 62: 3, 63: 1, 64: 1}
Verified via

didn't find the symbol for scenes
OVERALL indents: {25: 119, 5: 2226, 15: 1127, 20: 68}
0.0 0.6288135593220339
!!!Failed to split (with overall)
f4bd30511bb4dc1dd7dea595b86c6d466997cfe6
['                         ']
FADE IN indent: 25
SCENE indents: {10: 4137, 11: 23}
DIALOG indents: {36: 1, 35: 1, 58: 1}
0.0 0.9937545039634879
!!!Failed to split
f5255dcda3e92492cca0b95687bf01d0908b07b4
didn't find the symbol for scenes
OVERALL indents: {10: 2059, 6: 1, 20: 1896, 25: 138, 26: 35, 5: 1, 3: 4, 7: 9, 17: 6, 28: 1}
0.0 0.0009638554216867469
0.0009638554216867469 0.0012048192771084338
0.0012048192771084338 0.0014457831325301205
0.0014457831325301205 0.0036144578313253013
0.0036144578313253013 0.49975903614457834
0.49975903614457834 0.5012048192771085
0.5012048192771085 0.9580722891566265
split here
!!!Re-organized overall dicts:
SCENE indents: {3: 4, 5: 1, 6: 1, 7: 9, 10: 2059, 17: 6}
DIALOG indents: {20: 1896, 25: 138, 26: 35, 28: 1}
Verified via ratio split at 20
f5fb9cfd351645796e1120173

In [ ]:
# print('%d scripts satisfying the ratio constraints.' % num_verified)
# print('\tAmong them there are %d handled with FADE IN only.' % num_fade_in_success)
# print('%d scripts failed' % num_failed)
# print('%d total scripts' % num_movie)

300 scripts satisfying the ratio constraints.
	Among them there are 125 handled with FADE IN only.
121 scripts failed
433 total scripts


In [ ]:
#############################
##### Splitting scripts #####
#############################

count_movie = 0
count_meta_complete = 0
count_start_pattern = 0
count_summary = 0
if not os.path.exists('./preprocessed/'):
    os.mkdir('./preprocessed/')
if not os.path.exists('./preprocessed/movies/'):
    os.mkdir('./preprocessed/movies/')
    
count_with_fade_in = 0
list_num_scenes = []
list_num_dialogs = []

num_verified = 0
num_failed = 0
num_movie = 0
num_fade_in_success = 0

SPLIT_META = {}
meta_filein = open('preprocessed/split_meta_info.txt')
for line in meta_filein:
    meta_info = json.loads(line.strip())
    SPLIT_META[meta_info['id']] = meta_info
    
for each_id in STORY_META.keys():
    if each_id in exclusive_id:
        print(each_id, 'has some issues and will be skipped')
        continue
        
    if each_id in special_id:
        print(each_id, 'currently skipping the special script formats')
        continue
#     each_id = 'cd13eb380843224f66c34047cc06dc445a92f8fd'
#     each_id = '18bcdf16fb896faf7fb36db6b752f16f0343ad24'
#     each_id = '1508452704b5829941b98aeee2eeba192f5beae5'
#     each_id = '00ee9e01a0e581e0d8cbf7e865a895147c480c5e'
    print(each_id)
    if SPLIT_META[each_id]['type'] == 'Fail':
        continue
    elif SPLIT_META[each_id]['type'].startswith('Original'):
        fileout = open(os.path.join('preprocessed/silver', each_id + '.txt'), 'w', encoding='cp1256')
    elif SPLIT_META[each_id]['type'].startswith('Reorg'):
#         continue
        fileout = open(os.path.join('preprocessed/by_stats', each_id + '.txt'), 'w', encoding='cp1256')
        
    print(SPLIT_META[each_id])
        
    num_movie += 1
    
    num_scenes = 0
    num_dialogs = 0
    
    with open(os.path.join('tmp', each_id + '.content'), 'r', encoding='cp1256') as file:
        content = file.read()

    soup = BeautifulSoup(content, 'html.parser')
    
    bs = soup.pre.find_all('b')

#     result = [[element.string, element.next_sibling.string] for element in bs if (element.next_sibling and element.next_sibling.string is not None)]

    sents = [(element.string, [ele for ele in element.next_sibling.string.split('\n') if ele.strip()]) #split the text by \n; remove space
             for element in bs if (element.next_sibling and element.next_sibling.string is not None)] #select text 
    
    scene_symbol = None
    dialogue_symbol = None
    scene_found = False
    fade_in_indent = -1

    for element in bs:
#         print(element.string)
        if element is None or element.string is None:
            continue
        if not scene_found and element.string.find('FADE IN') != -1:#find if there is a FADE IN in the sentence
            scene_symbol = str(element.string)[:element.string.index('FADE IN')]#count the space before FADE IN
            scene_found = True#only need the 1st FADE IN (incase there is any FADE IN in the text)

        if scene_found and element.string.startswith(scene_symbol + scene_symbol):#find if there are texts start with 2* the space before FADE IN
            dialogue_symbol = scene_symbol + scene_symbol
            break

    if scene_symbol is None:# there is no FADE IN in the script
        print("didn't find the symbol for scenes")
    else:
        print([scene_symbol])
        
        pointer = 0
        if pointer != len(scene_symbol):
            while scene_symbol[pointer] == ' ' or scene_symbol[pointer] == '\t' or scene_symbol[pointer] == '\n':
                pointer += 1
                if pointer == len(scene_symbol):
                    break
        fade_in_indent = pointer
        print('FADE IN indent:', fade_in_indent)
#         fade_in_indent = len(scene_symbol)
        count_with_fade_in += 1
        
    scene_indent_dict = {}
    indent_dict = {}
    dialog_indent_dict = {}
    section_structures = []
    
    new_sections = []
#     num_movie += 1

    for section in sents:
#         print(section)
        if len(section[1]) == 0:
            continue
        if section[0] is not None:
            section_name = re.sub(r'\t', '', section[0]).strip()
        else:
            section_name = 'NULL'
    #     section_name = re.sub(r'\t', '', section[0]).strip()
        section_texts = ''

        prev_indent = -1
        new_scene = False
        
        for sent in section[1]:
            pointer = 0
            if pointer == len(sent):
    #             section_structures.append([section_name, '', ''])
                continue
            while sent[pointer] == ' ' or sent[pointer] == '\t' or sent[pointer] == '\n':
                pointer += 1
                if pointer == len(sent):
                    break

            if pointer == len(sent):
    #             section_structures.append([section_name, '', ''])
                continue

            indent = str(pointer)
#             print(indent)
                    
            if indent in SPLIT_META[each_id]['scene_dict'] and prev_indent in SPLIT_META[each_id]['dialog_dict']:
                new_sections.append(('DIALOG', section_name, section_texts))
                num_dialogs += 1
                section_name = 'NULL'
                section_texts = ''
                new_scene = True

            section_texts += re.sub(r'\t', '', sent).strip() + '\n'
            prev_indent = indent
               
        if new_scene:
            new_sections.append(('SCENE', section_name, section_texts))
            num_scenes += 1
        else:
            if indent in SPLIT_META[each_id]['dialog_dict']:
                new_sections.append(('DIALOG', section_name, section_texts))
                num_dialogs += 1
            elif indent in SPLIT_META[each_id]['scene_dict']:
                new_sections.append(('SCENE', section_name, section_texts))
                num_scenes += 1
            else:
                print('ERROR: unknown indent:', prev_indent)
#                 print(section)

    print('num_scenes:', num_scenes)
    print('num_dialogs:', num_dialogs)
    for new_section in new_sections:
    #     print(new_section)
        fileout.write(json.dumps(new_section))
        fileout.write('\n')
    fileout.close()
#     break


0025577043f5090cd603c6aea60f26e236195594
00ee9e01a0e581e0d8cbf7e865a895147c480c5e
{'id': '00ee9e01a0e581e0d8cbf7e865a895147c480c5e', 'fade-in symbol': ['                                                                    '], 'fade-in indent': 68, 'type': 'Reorg fade-in', 'indent_dict': {'39': 97, '30': 156, '59': 1, '68': 10, '14': 25, '65': 15, '10': 645, '77': 13, '15': 691, '16': 571, '82': 3, '38': 85, '32': 21, '28': 160, '67': 10, '80': 7, '71': 33, '76': 19, '25': 86, '35': 55, '37': 24, '29': 154, '36': 108, '66': 15, '72': 6, '75': 18, '33': 161, '70': 20, '27': 50, '23': 276, '34': 26, '11': 55, '26': 174, '40': 24, '78': 5, '74': 13, '17': 31, '73': 23, '31': 45, '21': 40, '20': 249, '62': 5, '24': 37, '63': 10, '69': 12, '61': 1, '60': 4, '42': 18, '43': 1, '79': 8, '41': 20, '84': 1, '83': 1, '47': 1, '12': 1, '18': 2, '22': 4}, 'scene_dict': {'10': 645, '11': 55, '12': 1, '14': 25, '15': 691, '16': 571, '17': 31, '18': 2, '20': 249, '21': 40, '22': 4, '23': 276}, 'dialog_

['']
FADE IN indent: 0
num_scenes: 254
num_dialogs: 955
0ffdd0e4abb67f0012a68d719a2d509ee8ec643d
{'id': '0ffdd0e4abb67f0012a68d719a2d509ee8ec643d', 'fade-in symbol': [], 'fade-in indent': -1, 'type': 'Reorg overall', 'indent_dict': {'0': 1725, '7': 31, '10': 1261, '14': 295, '15': 91, '13': 220, '17': 69, '18': 6, '16': 112, '12': 346, '8': 30, '11': 225, '9': 9, '6': 57, '3': 72, '20': 1, '19': 4, '22': 1, '5': 31}, 'scene_dict': {'0': 1725, '3': 72, '5': 31, '6': 57, '7': 31, '8': 30, '9': 9}, 'dialog_dict': {'10': 1261, '11': 225, '12': 346, '13': 220, '14': 295, '15': 91, '16': 112, '17': 69, '18': 6, '19': 4, '20': 1, '22': 1}}
didn't find the symbol for scenes
num_scenes: 934
num_dialogs: 978
10b8636838cdba5f0389d9d88b2ff83194491166
117ba539f2d311e3067064dca3b5fd68a7b95ee3
{'id': '117ba539f2d311e3067064dca3b5fd68a7b95ee3', 'fade-in symbol': [''], 'fade-in indent': 0, 'type': 'Original fade-in', 'indent_dict': {'9': 5, '5': 13, '0': 961, '2': 2632, '3': 136, '12': 1, '7': 14, '4':

num_scenes: 559
num_dialogs: 941
19dd268e42d372d73072c40bc3b8d1c46e07a3a2
{'id': '19dd268e42d372d73072c40bc3b8d1c46e07a3a2', 'fade-in symbol': [''], 'fade-in indent': 0, 'type': 'Original fade-in', 'indent_dict': {'0': 1492, '1': 1363, '4': 16, '2': 21}, 'scene_dict': {'0': 1492}, 'dialog_dict': {'1': 1363, '4': 16, '2': 21}}
['']
FADE IN indent: 0
num_scenes: 459
num_dialogs: 544
1a020bc35fe8ff93de0a570f7cf70e958880241c
{'id': '1a020bc35fe8ff93de0a570f7cf70e958880241c', 'fade-in symbol': [], 'fade-in indent': -1, 'type': 'Reorg overall', 'indent_dict': {'15': 1410, '30': 69, '25': 2471}, 'scene_dict': {'15': 1410}, 'dialog_dict': {'25': 2471, '30': 69}}
didn't find the symbol for scenes
num_scenes: 640
num_dialogs: 1052
1a447099edd43fe60f1bf8f2b1eabbd11a5c3989
{'id': '1a447099edd43fe60f1bf8f2b1eabbd11a5c3989', 'fade-in symbol': [''], 'fade-in indent': 0, 'type': 'Original fade-in', 'indent_dict': {'4': 1, '0': 1322, '17': 9, '10': 401, '12': 23, '54': 24, '13': 1775, '15': 17, '20': 2

num_scenes: 769
num_dialogs: 769
21813f2122fca16d05e89d44f4521f7da8a8f3b7
{'id': '21813f2122fca16d05e89d44f4521f7da8a8f3b7', 'fade-in symbol': [], 'fade-in indent': -1, 'type': 'Reorg overall', 'indent_dict': {'31': 2, '32': 5, '13': 1, '4': 55, '14': 32, '68': 22, '17': 130, '22': 16, '23': 60, '64': 6, '28': 1, '67': 28, '18': 577, '5': 704, '66': 29, '15': 371, '70': 45, '20': 89, '21': 34, '65': 13, '6': 1485, '16': 616, '52': 1, '69': 24, '24': 75, '71': 13, '59': 2, '7': 138, '19': 1162, '56': 4, '63': 3, '62': 3, '72': 4, '25': 19, '55': 2, '29': 1, '33': 5, '30': 5, '34': 2, '60': 3, '58': 1, '26': 2, '50': 1}, 'scene_dict': {'4': 55, '5': 704, '6': 1485, '7': 138, '13': 1, '14': 32, '15': 371, '16': 616}, 'dialog_dict': {'17': 130, '18': 577, '19': 1162, '20': 89, '21': 34, '22': 16, '23': 60, '24': 75, '25': 19, '26': 2, '28': 1, '29': 1, '30': 5, '31': 2, '32': 5, '33': 5, '34': 2, '50': 1, '52': 1, '55': 2, '56': 4, '58': 1, '59': 2, '60': 3, '62': 3, '63': 3, '64': 6, '65'

didn't find the symbol for scenes
num_scenes: 315
num_dialogs: 640
317fe46b2903da739b059733cd618db6ca49a494
{'id': '317fe46b2903da739b059733cd618db6ca49a494', 'fade-in symbol': [''], 'fade-in indent': 0, 'type': 'Original fade-in', 'indent_dict': {'0': 1088, '2': 2205, '3': 13}, 'scene_dict': {'0': 1088}, 'dialog_dict': {'2': 2205, '3': 13}}
['']
FADE IN indent: 0
num_scenes: 408
num_dialogs: 1118
324138e357569216a5fae897cea8cba4548626c4
{'id': '324138e357569216a5fae897cea8cba4548626c4', 'fade-in symbol': [], 'fade-in indent': -1, 'type': 'Reorg overall', 'indent_dict': {'38': 1, '32': 17, '31': 3, '29': 3, '0': 1960, '16': 1255, '27': 9, '26': 6, '25': 2, '30': 10, '21': 3, '14': 7, '28': 4, '20': 1, '23': 3, '24': 1, '19': 1}, 'scene_dict': {'0': 1960, '14': 7}, 'dialog_dict': {'16': 1255, '19': 1, '20': 1, '21': 3, '23': 3, '24': 1, '25': 2, '26': 6, '27': 9, '28': 4, '29': 3, '30': 10, '31': 3, '32': 17, '38': 1}}
didn't find the symbol for scenes
num_scenes: 530
num_dialogs: 718
3

...

didn't find the symbol for scenes
num_scenes: 545
num_dialogs: 1133
8dddfb1c4aa33c5821670ba20549ec02aba73056
{'id': '8dddfb1c4aa33c5821670ba20549ec02aba73056', 'fade-in symbol': [], 'fade-in indent': -1, 'type': 'Reorg overall', 'indent_dict': {'14': 2, '5': 1770, '15': 1421, '29': 22, '21': 118, '22': 1}, 'scene_dict': {'5': 1770, '14': 2}, 'dialog_dict': {'15': 1421, '21': 118, '22': 1, '29': 22}}
didn't find the symbol for scenes
num_scenes: 540
num_dialogs: 745
8e7d209dd54a50345632ec639627a23be362a95c
8efe7830fb894d0dc87cb56b37cd4ec44a7ec672
907e3c83c161299b8f178e98399d4e2cc0dcd57f
90c5a4e276e3f9dec41d7be579052b9327f56197
91075ab5383e8113d86b17536c6918bfbbd6af20
{'id': '91075ab5383e8113d86b17536c6918bfbbd6af20', 'fade-in symbol': [''], 'fade-in indent': 0, 'type': 'Reorg fade-in', 'indent_dict': {'28': 1, '21': 1, '16': 1, '5': 421, '0': 10, '31': 157, '19': 371, '26': 20, '25': 1, '27': 7, '1': 1, '55': 2}, 'scene_dict': {'0': 10, '1': 1, '5': 421, '16': 1}, 'dialog_dict': {'19': 

['          ']
FADE IN indent: 10
num_scenes: 543
num_dialogs: 906
9cd25b973d253386eaebbb7f2f7821dc7518f6d6
{'id': '9cd25b973d253386eaebbb7f2f7821dc7518f6d6', 'fade-in symbol': [], 'fade-in indent': -1, 'type': 'Reorg overall', 'indent_dict': {'19': 43, '23': 1, '18': 2, '35': 2, '0': 1497, '11': 444, '28': 2, '12': 56, '1': 73, '33': 1, '17': 28, '26': 9, '6': 1362, '20': 5, '7': 75, '10': 109, '8': 34, '16': 6, '9': 180, '15': 16, '21': 1, '14': 2, '4': 8, '5': 59, '47': 1}, 'scene_dict': {'0': 1497, '1': 73, '4': 8, '5': 59}, 'dialog_dict': {'6': 1362, '7': 75, '8': 34, '9': 180, '10': 109, '11': 444, '12': 56, '14': 2, '15': 16, '16': 6, '17': 28, '18': 2, '19': 43, '20': 5, '21': 1, '23': 1, '26': 9, '28': 2, '33': 1, '35': 2, '47': 1}}
didn't find the symbol for scenes
num_scenes: 683
num_dialogs: 891
9ce9cda3bf3ab43a4e0c0199110e30727770477c
9d8b79107628c4dedae91fa3648e56ae94b997fc
{'id': '9d8b79107628c4dedae91fa3648e56ae94b997fc', 'fade-in symbol': ['               '], 'fade-in 

['']
FADE IN indent: 0
num_scenes: 733
num_dialogs: 1263
a912a7308035d8c7136ca7439bce977ed57951f4
{'id': 'a912a7308035d8c7136ca7439bce977ed57951f4', 'fade-in symbol': [], 'fade-in indent': -1, 'type': 'Reorg overall', 'indent_dict': {'30': 1, '20': 60, '6': 1386, '16': 1786, '64': 4, '26': 8, '62': 7, '19': 97, '23': 5, '10': 2, '65': 2, '24': 3, '21': 5, '63': 2, '17': 18, '25': 1, '61': 2, '18': 9, '22': 1}, 'scene_dict': {'6': 1386, '10': 2}, 'dialog_dict': {'16': 1786, '17': 18, '18': 9, '19': 97, '20': 60, '21': 5, '22': 1, '23': 5, '24': 3, '25': 1, '26': 8, '30': 1, '61': 2, '62': 7, '63': 2, '64': 4, '65': 2}}
didn't find the symbol for scenes
num_scenes: 592
num_dialogs: 885
aa53a1867711a3dd8ff9f8f29266398e420813d3
aa70927925013f9111ea80c2ab81449c4ca3cbdf
acdeff3068372e962ae2600b4883e5cd62c04150
{'id': 'acdeff3068372e962ae2600b4883e5cd62c04150', 'fade-in symbol': [], 'fade-in indent': -1, 'type': 'Reorg overall', 'indent_dict': {'33': 1, '27': 1, '24': 1, '16': 1, '20': 1, '1'

didn't find the symbol for scenes
num_scenes: 591
num_dialogs: 911
b6ed74a969cdbb5ddc34575df8b7ee9d955d5556
{'id': 'b6ed74a969cdbb5ddc34575df8b7ee9d955d5556', 'fade-in symbol': [], 'fade-in indent': -1, 'type': 'Reorg overall', 'indent_dict': {'22': 52, '27': 4, '29': 5, '19': 193, '28': 24, '23': 16, '16': 170, '52': 1, '4': 36, '14': 41, '18': 1115, '66': 7, '11': 109, '17': 102, '21': 7, '67': 5, '12': 3, '5': 644, '65': 75, '15': 686, '68': 3, '25': 10, '13': 1, '62': 1, '61': 4, '20': 18, '63': 2, '31': 1, '69': 1, '10': 1, '26': 2, '64': 5, '6': 139, '30': 1}, 'scene_dict': {'4': 36, '5': 644, '6': 139, '10': 1, '11': 109, '12': 3, '13': 1, '14': 41, '15': 686, '16': 170, '17': 102}, 'dialog_dict': {'18': 1115, '19': 193, '20': 18, '21': 7, '22': 52, '23': 16, '25': 10, '26': 2, '27': 4, '28': 24, '29': 5, '30': 1, '31': 1, '52': 1, '61': 4, '62': 1, '63': 2, '64': 5, '65': 75, '66': 7, '67': 5, '68': 3, '69': 1}}
didn't find the symbol for scenes
num_scenes: 931
num_dialogs: 760

didn't find the symbol for scenes
num_scenes: 480
num_dialogs: 972
c29a505a7f2efe48bc73da66d4608888b8dc617b
c2e872845e99f82ddfa5a3ed84f985a04b6729e6
{'id': 'c2e872845e99f82ddfa5a3ed84f985a04b6729e6', 'fade-in symbol': ['\t\t'], 'fade-in indent': 2, 'type': 'Reorg fade-in', 'indent_dict': {'12': 1529, '22': 1845, '26': 56}, 'scene_dict': {'12': 1529}, 'dialog_dict': {'22': 1845, '26': 56}}
['\t\t']
FADE IN indent: 2
num_scenes: 669
num_dialogs: 1196
c3d05fedec86ea11bb70837d54b941589bde4d88
{'id': 'c3d05fedec86ea11bb70837d54b941589bde4d88', 'fade-in symbol': [], 'fade-in indent': -1, 'type': 'Reorg overall', 'indent_dict': {'34': 1, '44': 1, '28': 1, '1': 982, '2': 2556, '3': 152, '13': 1, '0': 1}, 'scene_dict': {'0': 1, '1': 982}, 'dialog_dict': {'2': 2556, '3': 152, '13': 1, '28': 1, '34': 1, '44': 1}}
didn't find the symbol for scenes
num_scenes: 407
num_dialogs: 1070
c495d58b38155f9d465fc570dd35a74e9551dcaf
c5260ebf086da66320320ca1bd1c15a456610b17
c5384ac7f6a3e69a17ede247235936b934a7

In [ ]:
print(count_with_fade_in)
print(list_num_scenes)
print(list_num_dialogs)

list_ratio_scenes = []
list_ratio_dialogs = []

for i, (x, y) in enumerate(zip(list_num_scenes, list_num_dialogs)):
    if x / (x+y) > 0.9 or x / (x+y) < 0.1:
        continue # skip the extreme cases
    list_ratio_scenes.append(x / (x+y))
    list_ratio_dialogs.append(y / (x+y))
print(len(list_ratio_scenes))

162
[4164, 1184, 3242, 1052, 925, 969, 2203, 3553, 3359, 1050, 3326, 1755, 3035, 2, 1513, 1884, 2050, 1122, 1522, 1948, 1419, 1392, 90, 1749, 110, 3398, 3166, 1505, 2081, 1382, 2870, 2112, 1546, 444, 3590, 1550, 1006, 4785, 2277, 496, 2962, 8198, 1596, 1862, 40, 1096, 963, 512, 2801, 546, 1722, 75, 1501, 1072, 2213, 3278, 186, 1445, 3115, 1225, 3235, 2972, 2878, 942, 1255, 1439, 3460, 3549, 1289, 3984, 1815, 989, 3302, 3099, 2488, 1330, 2759, 2020, 1089, 1368, 1238, 1631, 2724, 1343, 1575, 3978, 2864, 2994, 137, 3300, 569, 1042, 998, 2274, 871, 1409, 1220, 946, 1648, 1027, 428, 811, 939, 2961, 1664, 3, 1630, 2588, 3052, 1277, 0, 4596, 2559, 1715, 965, 3938, 3980, 778, 1206, 1043, 546, 3892, 3475, 0, 2638, 3214, 1801, 2050, 1759, 1406, 3056, 864, 4024, 1359, 1368, 908, 526, 3056, 3423, 860, 2426, 1239, 3230, 1347, 2702, 1861, 1891, 471, 1065, 2014, 1857, 4160, 759, 1226, 2898, 1540, 1346, 1063, 1304, 1251, 3178, 1408]
[182, 2297, 1, 3019, 2042, 1715, 1579, 1084, 0, 1307, 157, 2332, 186,

In [ ]:
# print(list_ratio_scenes)
# print(list_ratio_dialogs)

import numpy as np
print('mean num of scenes:', np.mean(list_ratio_scenes))
print('std num of scenes:', np.std(list_ratio_scenes))
print('mean num of dialogs:', np.mean(list_ratio_dialogs))
print('std num of dialogs:', np.std(list_ratio_dialogs))

mean num of scenes: 0.3870662796573337
std num of scenes: 0.13559559099176607
mean num of dialogs: 0.6129337203426664
std num of dialogs: 0.13559559099176607


In [ ]:

with open(os.path.join('tmp', 'ff53fd53a94f343b8365915645b79d7ad5b1528e.content'), 'r') as file:
    content = file.read()
    
soup = BeautifulSoup(content, 'html.parser')

#print(soup.prettify())
soup.pre


<pre><html>
<head>
<script>
<b><!--
</b>if (window!= top)
top.location.href=location.href
<b>// -->
</b></script>
<title>FIGHT CLUB</title>
</head>
<pre>


<b>     F I G H T   C L U B
</b>
     by Jim Uhls

     based on a novel by Chuck Palahnuik

<b>     2/16/98
</b>
<b>     --------------------------------------------------------------
</b>
<b>     SCREEN BLACK
</b>
<b>                             JACK (V.O.)
</b>                 People were always asking me, did I
                 know Tyler Durden.

<b>     FADE IN:
</b>
<b>     INT.  SOCIAL ROOM - TOP FLOOR OF HIGH RISE -- NIGHT
</b>
     TYLER has one arm around Jack's shoulder; the other hand
     holds a HANDGUN with the barrel lodged in JACK'S MOUTH.
     Tyler is sitting in Jack's lap.

     They are both sweating and disheveled, both around 30; Tyler
     is blond, handsome; and Jack, brunette, is appealing in a
     dry sort of way.  Tyler looks at his watch.

<b>                             TYLER
</b>                 One 

In [ ]:
# bs = soup.find_all('b')
bs = soup.pre.find_all('b')

result = [[element.string, element.next_sibling.string] for element in bs if element.next_sibling]

sents = [(element.string, [ele for ele in element.next_sibling.string.split('\n') if ele.strip()]) #split the text by \n; remove space
         for element in bs if element.next_sibling] #select text 

for sent in sents:
#     if 'Writers' in sent:
    print(sent)

('     F I G H T   C L U B\n', ['     by Jim Uhls', '     based on a novel by Chuck Palahnuik'])
('     2/16/98\n', [])
('     --------------------------------------------------------------\n', [])
('     SCREEN BLACK\n', [])
('                             JACK (V.O.)\n', ['                 People were always asking me, did I', '                 know Tyler Durden.'])
('     FADE IN:\n', [])
('     INT.  SOCIAL ROOM - TOP FLOOR OF HIGH RISE -- NIGHT\n', ["     TYLER has one arm around Jack's shoulder; the other hand", "     holds a HANDGUN with the barrel lodged in JACK'S MOUTH.", "     Tyler is sitting in Jack's lap.", '     They are both sweating and disheveled, both around 30; Tyler', '     is blond, handsome; and Jack, brunette, is appealing in a', '     dry sort of way.  Tyler looks at his watch.'])
('                             TYLER\n', ['                 One minute.', '                       (looking out window)', "                 This is the beginning.  We're at", '          

In [ ]:
#FADE IN align at scene；the space before dialogue >= 2* the space before FADE IN
scene_symbol = None
dialogue_symbol = None
scene_found = False

for element in bs:
    if not scene_found and element.string.find('FADE IN') != -1:#find if there is a FADE IN in the sentence
        scene_symbol = str(element.string)[:element.string.index('FADE IN')]#count the space before FADE IN
        scene_found = True#only need the 1st FADE IN (incase there is any FADE IN in the text)

    if scene_found and element.string.startswith(scene_symbol + scene_symbol):#find if there are texts start with 2* the space before FADE IN
        dialogue_symbol = scene_symbol + scene_symbol
        break

if scene_symbol is None:# there is no FADE IN in the script
    print("didn't find the symbol for scenes")
else:
    print([scene_symbol])
    print('FADE IN indent:', len(scene_symbol))
    fade_in_indent = len(scene_symbol)

# if scene_symbol is None:# there is no FADE IN in the script
#     print("didn't find the symbol for scenes")
# else:
#     print([scene_symbol])

# if dialogue_symbol is None:#the space before dialogue != 2* the space before FADE IN
#     print("didn't find the symbol for dialogue")
# else:
#     print([dialogue_symbol])

indent_dict = {}
section_structures = []

new_sections = []
num_scenes = 0
num_dialogs = 0

for section in sents:
    section_name = re.sub(r'\t', '', section[0]).strip()
#     section_name = re.sub(r'\t', '', section[0]).strip()
    section_texts = ''
#     print(section_name)
    
    prev_indent = -1
    new_scene = False
    for (sent_id, sent) in enumerate(section[1]):
        pointer = 0
        if pointer == len(sent):
#             section_structures.append([section_name, '', ''])
            continue
        while sent[pointer] == ' ' or sent[pointer] == '\t' or sent[pointer] == '\n':
            pointer += 1
            if pointer == len(sent):
                break
                
        if pointer == len(sent):
#             section_structures.append([section_name, '', ''])
            continue

        indent = pointer
#         if indent > 30:
#             print(section)
#         section_type = indent
#         section_content = section_content[pointer:]

        if indent not in indent_dict:
            indent_dict[indent] = 1
        else:
            indent_dict[indent] += 1
            
        if indent <= fade_in_indent and prev_indent > fade_in_indent:
            new_sections.append(('DIALOG', section_name, section_texts))
            num_dialogs += 1
            section_name = 'NULL'
            section_texts = ''
            new_scene = True
            
#         print(sent)
        section_texts += re.sub(r'\t', '', sent).strip() + '\n'
        prev_indent = indent
        
    if new_scene:
        new_sections.append(('SCENE', section_name, section_texts))
        num_scenes += 1
    else:
        if prev_indent > fade_in_indent:
            new_sections.append(('DIALOG', section_name, section_texts))
            num_dialogs += 1
        else:
            new_sections.append(('SCENE', section_name, section_texts))
            num_scenes += 1
        
print(indent_dict)

# fileout = open('./preprocessed/with_bs/' + 'ff53fd53a94f343b8365915645b79d7ad5b1528e' + '.txt', 'w')
# for new_section in new_sections:
# #     print(new_section)
#     fileout.write(json.dumps(new_section))
#     fileout.write('\n')
# fileout.close()
print('num_scenes:', num_scenes)
print('num_dialogs:', num_dialogs)

['     ']
FADE IN indent: 5
{5: 1408, 17: 2454, 23: 116, 24: 3, 30: 2, 31: 2, 29: 6, 19: 2, 18: 2, 27: 1, 28: 1, 25: 1, 48: 1}
num_scenes: 734
num_dialogs: 1301
